In [49]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import sqlite3
from datetime import datetime
import ccxt
from typing import Dict, List, Tuple


class ScalpingStrategy:
    def __init__(self, initial_capital: float = 50.0, commission_rate: float = 0.001):
        self.capital = initial_capital
        self.commission_rate = commission_rate
        self.position_size = 0
        self.in_position = False
        self.trades_db = "trades.db"
        self.setup_database()
        # Parámetros de Supertrain
        self.atr_period = 10
        self.atr_multiplier = 3
        # Parámetros de Estocástico
        self.stoch_length = 7
        self.stoch_smoothk = 3
        self.stoch_smoothd = 3

    def setup_database(self):
        conn = sqlite3.connect(self.trades_db)
        c = conn.cursor()
        c.execute("""CREATE TABLE IF NOT EXISTS trades
                    (timestamp TEXT, symbol TEXT, action TEXT, 
                     price REAL, quantity REAL, commission REAL,
                     profit_loss REAL, running_capital REAL)""")
        conn.commit()
        conn.close()

    def log_trade(self, symbol: str, action: str, price: float, quantity: float, 
                  commission: float, profit_loss: float):
        """Registra las operaciones en la base de datos"""
        conn = sqlite3.connect(self.trades_db)
        c = conn.cursor()
        
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        c.execute("""INSERT INTO trades 
                     (timestamp, symbol, action, price, quantity, commission, 
                      profit_loss, running_capital)
                     VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
                  (timestamp, symbol, action, price, quantity, commission, 
                   profit_loss, self.capital))
        
        conn.commit()
        conn.close()
        
    def analyze_results(self) -> Dict:
        conn = sqlite3.connect(self.trades_db)
        df_trades = pd.read_sql_query("SELECT * FROM trades", conn)
        conn.close()

        if len(df_trades) == 0:
            return {
                "total_trades": 0,
                "profitable_trades": 0,
                "losing_trades": 0,
                "total_profit": 0,
                "max_profit": 0,
                "max_loss": 0,
                "final_capital": self.capital,
                "return_pct": 0,
            }

        results = {
            "total_trades": len(df_trades),
            "profitable_trades": len(df_trades[df_trades["profit_loss"] > 0]),
            "losing_trades": len(df_trades[df_trades["profit_loss"] <= 0]),
            "total_profit": df_trades["profit_loss"].sum(),
            "max_profit": df_trades["profit_loss"].max(),
            "max_loss": df_trades["profit_loss"].min(),
            "final_capital": self.capital,
            "return_pct": ((self.capital - 50) / 50) * 100,
            "win_rate": len(df_trades[df_trades["profit_loss"] > 0])
            / len(df_trades)
            * 100,
        }

        print("\n📊 Resumen del Backtest:")
        print(f"Total operaciones: {results['total_trades']}")
        print(f"Operaciones ganadoras: {results['profitable_trades']}")
        print(f"Operaciones perdedoras: {results['losing_trades']}")
        print(f"Win rate: {results['win_rate']:.2f}%")
        print(f"Beneficio total: {results['total_profit']:.2f} USDT")
        print(f"Máximo beneficio: {results['max_profit']:.2f} USDT")
        print(f"Máxima pérdida: {results['max_loss']:.2f} USDT")
        print(f"Capital final: {results['final_capital']:.2f} USDT")
        print(f"Retorno: {results['return_pct']:.2f}%")

        return results

    def calculate_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        # Cálculo del Supertrain (usando ATR como base)
        df["atr"] = ta.atr(df["high"], df["low"], df["close"], length=self.atr_period)
        df["supertrain_upper"] = df["close"] + (df["atr"] * self.atr_multiplier)
        df["supertrain_lower"] = df["close"] - (df["atr"] * self.atr_multiplier)
        
        # Cálculo del Estocástico
        stoch = ta.stoch(df["high"], df["low"], df["close"], 
                        k=self.stoch_length, 
                        d=self.stoch_smoothk, 
                        smooth_k=self.stoch_smoothd)
        df["stoch_k"] = stoch["STOCHk_7_3_3"]
        df["stoch_d"] = stoch["STOCHd_7_3_3"]
        
        # Determinar tendencia con Supertrain
        df["trend"] = "neutral"
        df.loc[df["close"] > df["supertrain_upper"], "trend"] = "bullish"
        df.loc[df["close"] < df["supertrain_lower"], "trend"] = "bearish"
        
        return df

    def identify_market_condition(self, row: pd.Series) -> str:
        """Identifica el estado actual del mercado basado en Supertrain y Estocástico"""
        
        # Determinar tendencia usando Supertrain
        trend = row["trend"]
        
        # Determinar condición del estocástico
        stoch_condition = "neutral"
        if row["stoch_k"] > 80:
            stoch_condition = "overbought"
        elif row["stoch_k"] < 20:
            stoch_condition = "oversold"
            
        # Determinar momentum basado en el cruce del estocástico
        momentum = "neutral"
        if row["stoch_k"] > row["stoch_d"]:
            momentum = "bullish"
        elif row["stoch_k"] < row["stoch_d"]:
            momentum = "bearish"
        
        return f"{trend}_{stoch_condition}_{momentum}"


    def get_signal(self, row: pd.Series) -> Tuple[str, float, float]:
        """
        Genera señales basadas en Supertrain y Estocástico
        Returns: (signal, stop_loss, take_profit)
        """
        signal = "HOLD"
        stop_loss = 0.0
        take_profit = 0.0

        # Zonas de sobrecompra/sobreventa
        overbought_level = 80
        oversold_level = 20

        # Señales de compra
        if row["trend"] == "bullish" and not self.in_position:
            # Cruce alcista en zona de sobreventa
            if (row["stoch_k"] > row["stoch_d"] and 
                row["stoch_k"] < oversold_level):
                signal = "BUY"
                stop_loss = row["supertrain_lower"]
                # Take profit con ratio 1:2
                risk = row["close"] - stop_loss
                take_profit = row["close"] + (risk * 2)

        # Señales de venta
        elif row["trend"] == "bearish" and not self.in_position:
            # Cruce bajista en zona de sobrecompra
            if (row["stoch_k"] < row["stoch_d"] and 
                row["stoch_k"] > overbought_level):
                signal = "SELL"
                stop_loss = row["supertrain_upper"]
                # Take profit con ratio 1:2
                risk = stop_loss - row["close"]
                take_profit = row["close"] - (risk * 2)

        return signal, stop_loss, take_profit

    def calculate_position_size(self, price: float, stop_loss: float) -> float:
        """Calcula el tamaño de la posición con gestión de riesgo estricta"""
        risk_per_trade = self.capital * 0.01  # 1% riesgo máximo por operación
        risk_per_unit = price - stop_loss

        if risk_per_unit <= 0:
            return 0

        position_size = risk_per_trade / risk_per_unit
        max_position = (self.capital * 0.95) / price  # Máximo 95% del capital
        return min(position_size, max_position)

    def execute_backtest(self, df: pd.DataFrame, symbol: str = "BTC/USDT"):
        df = self.calculate_indicators(df)
        trades = []
        self.entry_price = 0
        self.stop_loss_price = 0
        self.take_profit_price = 0
        consecutive_losses = 0
        max_consecutive_losses = 3

        print(f"🚀 Iniciando backtest con capital inicial: {self.capital} USDT")

        for index, row in df.iterrows():
            if consecutive_losses >= max_consecutive_losses:
                print("⚠️ Máximo de pérdidas consecutivas alcanzado. Deteniendo trading.")
                break

            signal, stop_loss, take_profit = self.get_signal(row)

            if signal == "BUY" and not self.in_position:
                self.position_size = self.calculate_position_size(row["close"], stop_loss)
                if self.position_size > 0:
                    commission = row["close"] * self.position_size * self.commission_rate
                    if commission + (row["close"] * self.position_size) <= self.capital:
                        self.entry_price = row["close"]
                        self.stop_loss_price = stop_loss
                        self.take_profit_price = take_profit
                        self.capital -= row["close"] * self.position_size + commission
                        self.in_position = True
                        self.log_trade(
                            symbol,
                            "BUY",
                            row["close"],
                            self.position_size,
                            commission,
                            0,
                        )
                        print(
                            f"🔵 COMPRA: Precio={row['close']:.2f} | "
                            f"Cantidad={self.position_size:.6f} | "
                            f"Capital={self.capital:.2f}"
                        )

            elif signal == "SELL" and self.in_position:
                commission = row["close"] * self.position_size * self.commission_rate
                profit_loss = (row["close"] - self.entry_price) * self.position_size - commission
                self.capital += row["close"] * self.position_size - commission
                self.log_trade(
                    symbol,
                    "SELL",
                    row["close"],
                    self.position_size,
                    commission,
                    profit_loss,
                )

                if profit_loss < 0:
                    consecutive_losses += 1
                else:
                    consecutive_losses = 0

                print(
                    f"🔴 VENTA: Precio={row['close']:.2f} | "
                    f"P/L={profit_loss:.2f} | Capital={self.capital:.2f}"
                )
                self.in_position = False
                self.position_size = 0

        return self.analyze_results()


# Ejemplo de uso
if __name__ == "__main__":
    exchange = ccxt.binance({"enableRateLimit": True})

    ohlcv = exchange.fetch_ohlcv("BTC/USDT", "5m", limit=5000)
    df = pd.DataFrame(
        ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
    )
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)

    strategy = ScalpingStrategy(initial_capital=50.0)
    results = strategy.execute_backtest(df)


🚀 Iniciando backtest con capital inicial: 50.0 USDT


In [ ]:
# Ejemplo de uso
if __name__ == "__main__":
    # Obtener datos históricos
    exchange = ccxt.binance({"enableRateLimit": True})

    # Obtenemos más datos históricos para mejor análisis
    ohlcv = exchange.fetch_ohlcv("BTC/USDT", "5m", limit=1000)
    df = pd.DataFrame(
        ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
    )
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)

    # Ejecutar estrategia
    strategy = ScalpingStrategy(initial_capital=50.0)
    results = strategy.execute_backtest(df)

In [ ]:
# Ejemplo de uso
if __name__ == "__main__":
    # Obtener datos históricos (ejemplo con ccxt)
    exchange = ccxt.binance({"enableRateLimit": True})

    ohlcv = exchange.fetch_ohlcv("BTC/USDT", "5m", limit=500)
    df = pd.DataFrame(
        ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
    )
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)


In [30]:
from pygold.download import descargar_datos_yahoo

In [35]:
df = descargar_datos_yahoo(
    ticker="BTC-USD", inicio="2024-11-01", fin="2024-11-09", intervalo="1m"
)

[*********************100%***********************]  1 of 1 completed


In [36]:
df

,open,high,low,close,adj close,volume
time,,,,,,
2024-11-01 00:00:00+00:00,70202.875000,70202.875000,70202.875000,70202.875000,70202.875000,0
2024-11-01 00:01:00+00:00,70208.898438,70208.898438,70208.898438,70208.898438,70208.898438,12374016
2024-11-01 00:03:00+00:00,70223.117188,70223.117188,70223.117188,70223.117188,70223.117188,46391296
2024-11-01 00:04:00+00:00,70241.531250,70241.531250,70241.531250,70241.531250,70241.531250,0
2024-11-01 00:05:00+00:00,70245.648438,70245.648438,70245.648438,70245.648438,70245.648438,0
...,...,...,...,...,...,...
2024-11-08 23:52:00+00:00,76533.835938,76533.835938,76533.835938,76533.835938,76533.835938,0
2024-11-08 23:53:00+00:00,76540.476562,76540.476562,76540.476562,76540.476562,76540.476562,0
2024-11-08 23:54:00+00:00,76544.757812,76544.757812,76544.757812,76544.757812,76544.757812,3166208


In [37]:
df = (
    df.reset_index()
    .rename(
        columns=dict(
            zip(
                ["time", "open", "high", "low", "close", "volume"],
                ["timestamp", "open", "high", "low", "close", "volume"],
            )
        )
    )
    .set_index("timestamp")
)

In [38]:
# Ejecutar estrategia
strategy = ScalpingStrategy(initial_capital=50.0)
results = strategy.execute_backtest(df)

🚀 Iniciando backtest con capital inicial: 50.0 USDT

🔍 Análisis de entrada:
Momentum positivo: False
Tendencia alineada: False
MACD favorable: False
RSI favorable: False
Precio actual: 70202.88
❌ No hay señal de compra - 0/4 condiciones cumplidas

🔍 Análisis de entrada:
Momentum positivo: False
Tendencia alineada: False
MACD favorable: False
RSI favorable: False
Precio actual: 70208.90
❌ No hay señal de compra - 0/4 condiciones cumplidas

🔍 Análisis de entrada:
Momentum positivo: False
Tendencia alineada: False
MACD favorable: False
RSI favorable: False
Precio actual: 70223.12
❌ No hay señal de compra - 0/4 condiciones cumplidas

🔍 Análisis de entrada:
Momentum positivo: False
Tendencia alineada: False
MACD favorable: False
RSI favorable: False
Precio actual: 70241.53
❌ No hay señal de compra - 0/4 condiciones cumplidas

🔍 Análisis de entrada:
Momentum positivo: False
Tendencia alineada: False
MACD favorable: False
RSI favorable: False
Precio actual: 70245.65
❌ No hay señal de compra -